In [1]:
l = [1,2,3,[4,5,6],6,[8,9],10]

In [9]:
for i in l:
    try:
        x = len(i)
        print(i)
    except TypeError:
        pass

[4, 5, 6]
[8, 9]


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn import model_selection
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import RobustScaler
# from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
# from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, \
                             GradientBoostingClassifier, \
                             RandomForestClassifier, \
                             BaggingClassifier,\
                             StackingClassifier, \
                             VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_train = pd.read_csv('train_dataset_train.csv', index_col='id')
df_test = pd.read_csv('test_dataset_test.csv', index_col='id')
# df = pd.concat([df_train, df_test])

In [5]:
df_train

,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
id,,,,,,,,,,,
1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017
...,...,...,...,...,...,...,...,...,...,...,...
1091017,6A738232F8E43DFCD4027FDC0EA6CC03,Пропуск FacePay,2283,Нижегородская (второй),15005,Нижегородская,15,Некрасовская,2022-09-19 02:54:50,124.750000,15005
1091018,141174B366F14A8703903813A49374DB,Пропуск FacePay,278,Калужская (Северный),6007,Калужская,6,Калужско-Рижская,2022-09-19 03:06:02,56.066667,6007
1091019,E38493B30A7C55170806F187AF5EEA15,Пропуск FacePay,118,Каширская (Северный ),2010,Каширская,2,Замоскворецкая,2022-09-19 03:17:00,91.283333,2010


In [6]:
df_train['ticket_type_nm'].unique().tolist()

['Пропуск FacePay',
 'СК учащегося 30 дней',
 'БСК дружинника г.Москвы',
 '30 дней',
 'КОШЕЛЕК',
 'СК студента 30 дней',
 '90 дней',
 '90 дней ЕДИНЫЙ ТК',
 '30 дней СК учащегося',
 '90 дней СК студента',
 '30 дней СК студента',
 'ББК',
 'СК аспиранта 90 дней',
 'Социальная карта москвича',
 '30 дней ЕДИНЫЙ ТК',
 'Социальная карта жителя Моск. области',
 '30 дней Пригород ТК',
 'Социальная карта москвича с сопровождающим',
 'Пропуск сотрудника УВД по охране Метрополитена',
 '60 поездок ЕДИНЫЙ ТК',
 'СК для сотрудника ГУВД г.Москвы',
 '365 дней ЕДИНЫЙ ТК',
 'ВЕСБ МОСКВА (7мин)',
 'СК студента 90 дней',
 '90 дней Пригород ТК',
 '30 дней  Пригород',
 '365 дней Пригород ТК',
 '365 дней',
 'СК ординатора 30 дней',
 '30 дней СК аспиранта',
 '30 дней СК ординатора',
 '90 дней СК ординатора',
 'ВЕСБ МО (7 мин)',
 'ВЕСБ МО (тип 5)',
 'СК ординатора 90 дней',
 'СК аспиранта 30 дней',
 '90 дней СК учащегося',
 'Временный билет ММ',
 'Социальная карта жителя Моск. области с сопровождающим',
 'СК уч

In [ ]:
df_train['unlimited_ticket'] = df_train.loc[df_train['ticket_type_nm'].isin(['СК учащегося 30 дней',
 'БСК дружинника г.Москвы',
 '30 дней',
 'СК студента 30 дней',
 '90 дней',
 '90 дней ЕДИНЫЙ ТК',
 '30 дней СК учащегося',
 '90 дней СК студента',
 '30 дней СК студента',
 'СК аспиранта 90 дней',
 'Социальная карта москвича',
 '30 дней ЕДИНЫЙ ТК',
 'Социальная карта жителя Моск. области',
 '30 дней Пригород ТК',
 'Социальная карта москвича с сопровождающим',
 'Пропуск сотрудника УВД по охране Метрополитена',
 'СК для сотрудника ГУВД г.Москвы',
 '365 дней ЕДИНЫЙ ТК',
 'СК студента 90 дней',
 '90 дней Пригород ТК',
 '30 дней  Пригород',
 '365 дней Пригород ТК',
 '365 дней',
 'СК ординатора 30 дней',
 '30 дней СК аспиранта',
 '30 дней СК ординатора',
 '90 дней СК ординатора',
 'СК ординатора 90 дней',
 'СК аспиранта 30 дней',
 '90 дней СК учащегося',
 'Временный билет ММ',
 'Социальная карта жителя Моск. области с сопровождающим',
 'СК учащегося 90 дней',
 '3 суток Единый ТК',
 'Пропуск сотрудника  метрополитена',
 '90 дней  Пригород',
 '1 сутки ЕДИНЫЙ ТК',
 'Пропуск руководителя метрополитена',
 'Пропуск  сотрудника МЦК',
 '90 дней СК аспиранта',
 '365 дней  Пригород',
 '1 сутки Пригород ТК',
 'СК ассистента-стажера 30 дней',
 'СК ассистента-стажера 90 дней',
 'Билет 3 суток ЕДИНЫЙ',
 '30 дней СК ассистента-стажера',
 'Билет 1 сутки ЕДИНЫЙ'])]

In [ ]:
df_train.to_csv('train_dataset_cleaned')

In [81]:
df_train.columns

Index(['ticket_id', 'ticket_type_nm', 'entrance_id', 'entrance_nm',
       'station_id', 'station_nm', 'line_id', 'line_nm', 'pass_dttm',
       'time_to_under', 'label'],
      dtype='object')

In [ ]:
df_train = pd.DataFrame(pd.get_dummies(data = df_train, columns = ['ticket_type_nm']))

In [ ]:
X_train = df_train.drop(['ticket_id', 'entrance_nm', 'station_nm', 'line_nm', 'time_to_under', 'label'], axis = 1)

In [ ]:
y_train = df_train['label']

In [ ]:
X_train